In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
import pandas as pd
from pyspark.sql import functions as f

In [ ]:
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]").set("spark.driver.memory", "4g"))
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .getOrCreate()
spark

In [ ]:
# rdd_csv = sc.wholeTextFiles('/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/csv')
# rdd_csv.collect()

rdd_json = sc.wholeTextFiles('/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/json')
rdd_json.collect()

# rdd_parquet = sc.wholeTextFiles('/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/parquet')
# rdd_parquet.collect()

[('file:/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/json/part-00007-44d37ba9-fccd-40f4-b939-d8b47af99c95-c000.json',
  '{"contract_id":"10_01_147_CHR49","target_local_start_timestamp":"2022-12-01T00:00:00.000Z","target_local_end_timestamp":"2023-02-12T00:00:00.000Z","tariff_name":"Eco Electric","charge_type":"buy","price":0.2041}\n{"contract_id":"10_01_147_CHR49","target_local_start_timestamp":"2023-02-12T00:00:00.000Z","target_local_end_timestamp":"2024-05-01T00:00:00.000Z","tariff_name":"Business Electric","charge_type":"buy","price":0.4312}\n{"contract_id":"10_01_147_CHR49","target_local_start_timestamp":"2024-05-01T00:00:00.000Z","target_local_end_timestamp":"2024-08-10T00:00:00.000Z","tariff_name":"Electric Pro","charge_type":"buy","price":0.2472}\n{"contract_id":"10_01_147_CHR49","target_local_start_timestamp":"2024-08-10T00:00:00.000Z","target_local_end_timestamp":"2024-09-18T00:00:00.000Z","tariff_name":"Business Electric","charge_type":"buy","price":0.254}\n

In [ ]:
# df_csv = spark.read.format('csv') \
#     .option('header', 'true') \
#     .option('inferSchema', 'true') \
#     .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/csv')
# df_csv.show()

df_json = spark.read.format('json') \
    .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/json')
df_json.show()

# df_parquet = spark.read.format('parquet') \
#     .option('header', 'true') \
#     .option('inferSchema', 'true') \
#     .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/parquet')
# df_parquet.show()

+--------------------+-------------------+--------------------+--------------------+------------+
|         annotations|        contract_id|           timestamp|               value|value_source|
+--------------------+-------------------+--------------------+--------------------+------------+
|{"region":"Europe...| 04_02_111 _ CHR12 |2023-01-01T06:00:...| 0.02591860654732236| measurement|
|{"region":"Europe...| 04 _02_111 _CHR12 |2023-01-01T17:00:...| 0.07385444264936832| measurement|
|{"region":"Europe...| 04_02_111 _ CHR12 |2023-01-01T17:30:...| 0.08180149515221906| measurement|
|{"region":"Europe...| 04 _02_111 _CHR12 |2023-01-01T21:30:...| 0.08670661371854547| measurement|
|{"region":"Europe...|04 _ 02 _111_CHR12 |2023-01-02T00:30:...| 0.03597601881331959| measurement|
|{"region":"Europe...|04 _02_111 _ CHR12 |2023-01-02T05:30:...| 0.03638379308965683| measurement|
|{"region":"Europe...|04 _02 _111 _CHR12 |2023-01-03T10:45:...|            0.931575| measurement|
|{"region":"Europe..

In [ ]:
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('value', T.DoubleType(), True),
    T.StructField('value_source', T.StringType(), False),
    T.StructField('annotations', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('/content/drive/MyDrive/Colab Notebooks/project/raw_time_series/json')
df_raw_time_series = spark.createDataFrame(df_raw_time_series.rdd, schema=data_schema)
df_raw_time_series.printSchema()
df_raw_time_series.show()

root
 |-- contract_id: string (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- value: double (nullable = true)
 |-- value_source: string (nullable = false)
 |-- annotations: string (nullable = true)

+-------------------+-------------------+--------------------+------------+--------------------+
|        contract_id|          timestamp|               value|value_source|         annotations|
+-------------------+-------------------+--------------------+------------+--------------------+
| 04_02_111 _ CHR12 |2023-01-01 06:00:00| 0.02591860654732236| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 17:00:00| 0.07385444264936832| measurement|{"region":"Europe...|
| 04_02_111 _ CHR12 |2023-01-01 17:30:22| 0.08180149515221906| measurement|{"region":"Europe...|
| 04 _02_111 _CHR12 |2023-01-01 21:30:00| 0.08670661371854547| measurement|{"region":"Europe...|
|04 _ 02 _111_CHR12 |2023-01-02 00:30:00| 0.03597601881331959| measurement|{"region":"Europe...|
|04 

In [ ]:
data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('target_local_start_timestamp', T.TimestampType(), False),
    T.StructField('target_local_end_timestamp', T.TimestampType(), False),
    T.StructField('tariff_name', T.StringType(), True),
    T.StructField('charge_type', T.StringType(), False),
    T.StructField('price', T.DoubleType(), True)
])

df_customer_tariff = spark.read.format('json') \
    .schema(data_schema) \
    .load('/content/drive/MyDrive/Colab Notebooks/project/customer_tariff/json')
df_customer_tariff = spark.createDataFrame(df_customer_tariff.rdd, schema=data_schema)
df_customer_tariff.printSchema()
df_customer_tariff.show()

root
 |-- contract_id: string (nullable = false)
 |-- target_local_start_timestamp: timestamp (nullable = false)
 |-- target_local_end_timestamp: timestamp (nullable = false)
 |-- tariff_name: string (nullable = true)
 |-- charge_type: string (nullable = false)
 |-- price: double (nullable = true)

+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|    contract_id|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|charge_type| price|
+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|04_02_111_CHR28|         2022-12-01 00:00:00|       2023-02-16 00:00:00|     Electric Pro|        buy| 0.302|
|04_02_111_CHR28|         2023-02-16 00:00:00|       2024-05-19 00:00:00| Electric Loyalty|        buy|0.3938|
|04_02_111_CHR28|         2024-05-19 00:00:00|       2024-08-31 00:00:00|     Eco Electric|        buy|0.2095|
|04_02_111_CHR28|         2024-08-

Cleaning the "contract_id" column of additional spaces

In [ ]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('contract_id', f.regexp_replace(f.col('contract_id'), ' ', ''))
)
df_raw_time_series.show()

+---------------+-------------------+--------------------+------------+--------------------+
|    contract_id|          timestamp|               value|value_source|         annotations|
+---------------+-------------------+--------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-01 06:00:00| 0.02591860654732236| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:00:00| 0.07385444264936832| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:30:22| 0.08180149515221906| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 21:30:00| 0.08670661371854547| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 00:30:00| 0.03597601881331959| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 05:30:00| 0.03638379308965683| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 10:45:00|            0.931575| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 18:30:11| 0.08816273670606922| measurement

Setting the "value_source" column with the value "missing" when the value is missing.

In [ ]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('value_source', f.when(f.col('value').isNull(), 'missing').otherwise(f.col('value_source')))
)
df_raw_time_series.show()

+---------------+-------------------+--------------------+------------+--------------------+
|    contract_id|          timestamp|               value|value_source|         annotations|
+---------------+-------------------+--------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-01 06:00:00| 0.02591860654732236| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:00:00| 0.07385444264936832| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:30:22| 0.08180149515221906| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 21:30:00| 0.08670661371854547| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 00:30:00| 0.03597601881331959| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 05:30:00| 0.03638379308965683| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 10:45:00|            0.931575| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 18:30:11| 0.08816273670606922| measurement

The "timestamp" column has small variations that need to be rectified. The meters measure the consumption every 15 minutes exactly starting at 00:00, but due to the processing, variations can occur at the time they send.
- Hint: find the closest multiple of 15 minutes and remove the seconds


In [ ]:
# minute check-up
df_raw_time_series.filter(~f.minute(f.col('timestamp')).isin([0, 15, 30, 45])).show()

+---------------+-------------------+-------------------+------------+--------------------+
|    contract_id|          timestamp|              value|value_source|         annotations|
+---------------+-------------------+-------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-09 18:43:25|0.07931026171375788| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-19 20:28:10| 0.1306192739425649| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-24 07:28:17|               NULL|     missing|{"region":"Europe...|
|04_02_111_CHR12|2023-01-25 06:28:02|0.01334198716674829| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-25 20:13:15| 0.1367427521665754| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-27 05:13:11|0.03411513849879192| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-28 15:28:11|               NULL|     missing|{"region":"Europe...|
|04_02_111_CHR12|2023-01-31 18:58:28| 0.1784215615663938| measurement|{"region":

In [ ]:
# We use unix_timestamp and round at minutes * seconds, before we multiply it back
df_raw_time_series = df_raw_time_series \
    .withColumn('timestamp',
                f.from_unixtime(f.round(f.unix_timestamp('timestamp') / (15 * 60)) * (15 * 60))) \
    .withColumn('timestamp',f.date_format('timestamp', 'yyyy-MM-dd HH:mm'))

df_raw_time_series.show()

+---------------+----------------+--------------------+------------+--------------------+
|    contract_id|       timestamp|               value|value_source|         annotations|
+---------------+----------------+--------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-01 06:00| 0.02591860654732236| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:00| 0.07385444264936832| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 17:30| 0.08180149515221906| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 21:30| 0.08670661371854547| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 00:30| 0.03597601881331959| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 05:30| 0.03638379308965683| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 10:45|            0.931575| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 18:30| 0.08816273670606922| measurement|{"region":"Europe...|
|04_02_111

In [ ]:
# check-up so the values are rounded
df_raw_time_series.filter(~f.minute('timestamp').isin([0, 15, 30, 45])).show()

+-----------+---------+-----+------------+-----------+
|contract_id|timestamp|value|value_source|annotations|
+-----------+---------+-----+------------+-----------+
+-----------+---------+-----+------------+-----------+

